In [15]:
from jobs import Sparkinit
from json import dump, dumps
from utils import formatar_sql
from pyspark.sql.types import StructType, StructField, TimestampType, LongType, StringType, DecimalType, BinaryType
spark_start = Sparkinit()

spark = spark_start.buscar_sessao_spark()

print(f"WebUI SparkJobs: {spark.sparkContext.uiWebUrl}")
spark.getActiveSession()

WebUI SparkJobs: http://BHS-NOTE188:4040


# Vendas

In [24]:
import os
# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
path = os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\desafio_panvel-data_engineer\datalake\camada_0_transient\VENDAS")
dados = spark.read.format("parquet").load(path)

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)

base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final

f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM vendas_tmp
"""


root
 |-- d_dt_vd: timestamp (nullable = true)
 |-- n_id_fil: long (nullable = true)
 |-- n_id_vd_fil: long (nullable = true)
 |-- v_cli_cod: string (nullable = true)
 |-- n_vlr_tot_vd: decimal(18,6) (nullable = true)
 |-- n_vlr_tot_desc: decimal(14,4) (nullable = true)
 |-- v_cpn_eml: string (nullable = true)
 |-- tp_pgt: string (nullable = true)

-RECORD 0------------------------------
 d_dt_vd        | 2023-10-12 21:00:00  
 n_id_fil       | 2356284              
 n_id_vd_fil    | 34366442231          
 v_cli_cod      | 016E6FCC4F98832719BC 
 n_vlr_tot_vd   | 55.960000            
 n_vlr_tot_desc | 13.9900              
 v_cpn_eml      | NAO                  
 tp_pgt         | A VISTA              
only showing top 1 row



"/* 'Bronze' */\nSELECT \n        d_dt_vd,\n\tn_id_fil,\n\tn_id_vd_fil,\n\tv_cli_cod,\n\tn_vlr_tot_vd,\n\tn_vlr_tot_desc,\n\tv_cpn_eml,\n\ttp_pgt,\n        BASE64(CONCAT(CAST(d_dt_vd AS STRING) , CAST(n_id_fil AS STRING) , CAST(n_id_vd_fil AS STRING) , CAST(v_cli_cod AS STRING) , CAST(n_vlr_tot_vd AS STRING) , CAST(n_vlr_tot_desc AS STRING) , CAST(v_cpn_eml AS STRING) , CAST(tp_pgt AS STRING))) AS hash_id\nFROM vendas_tmp\n"

In [3]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("vendas_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    COALESCE(date_format(vt.d_dt_vd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_emissao,
    CAST(vt.n_id_fil AS BIGINT) AS codigo_filial,
    CAST(vt.n_id_vd_fil AS BIGINT) AS id_venda_filial,
    COALESCE(CAST(vt.v_cli_cod AS STRING), '') AS codigo_cliente,
    CAST(vt.n_vlr_tot_vd AS DECIMAL(38, 2)) AS valor_total_venda,
    CAST(vt.n_vlr_tot_desc AS DECIMAL(38, 2)) AS valor_total_desconto,
    CASE 
        WHEN vt.v_cpn_eml  = 'SIM' THEN True
        ELSE False
    END AS enviado_email,
    COALESCE(CAST(vt.tp_pgt AS STRING), '') AS tipo_pagamento
    
FROM vendas_tmp as vt""")

spark.sql(query).show()
query

+-------------------+-------------+---------------+--------------------+-----------------+--------------------+-------------+--------------+
|       data_emissao|codigo_filial|id_venda_filial|      codigo_cliente|valor_total_venda|valor_total_desconto|enviado_email|tipo_pagamento|
+-------------------+-------------+---------------+--------------------+-----------------+--------------------+-------------+--------------+
|2023-10-12 21:00:00|      2356284|    34366442231|016E6FCC4F98832719BC|            55.96|               13.99|        false|       A VISTA|
|2023-10-12 21:00:00|      2221184|    35550863931|035D148EADC74B6C6D2F|            31.48|               25.21|        false|              |
|2023-10-27 21:00:00|      2188984|    37392732531|030C1011214A3317E850|             6.49|               13.14|        false|       A VISTA|
|2023-10-08 21:00:00|      2608284|     3672652731|04710AFAF1FD9C48EBC3|            52.99|               40.27|        false|              |
|2023-10-09 2

"/* 'Silver' */\nSELECT COALESCE(date_format(vt.d_dt_vd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_emissao,\n       CAST(vt.n_id_fil AS BIGINT) AS codigo_filial,\n       CAST(vt.n_id_vd_fil AS BIGINT) AS id_venda_filial,\n       COALESCE(CAST(vt.v_cli_cod AS STRING), '') AS codigo_cliente,\n       CAST(vt.n_vlr_tot_vd AS DECIMAL(38, 2)) AS valor_total_venda,\n       CAST(vt.n_vlr_tot_desc AS DECIMAL(38, 2)) AS valor_total_desconto,\n       CASE\n           WHEN vt.v_cpn_eml = 'SIM' THEN TRUE\n           ELSE FALSE\n       END AS enviado_email,\n       COALESCE(CAST(vt.tp_pgt AS STRING), '') AS tipo_pagamento\nFROM vendas_tmp AS vt"

# Pedidos

In [23]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\desafio_panvel-data_engineer\datalake\camada_0_transient\PEDIDOS"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final

f"""/* 'Bronze' */
SELECT 
        {colunas}, 
        {base_final}
FROM pedidos_tmp
"""

root
 |-- n_id_pdd: long (nullable = true)
 |-- d_dt_eft_pdd: date (nullable = true)
 |-- d_dt_entr_pdd: timestamp (nullable = true)
 |-- v_cnl_orig_pdd: string (nullable = true)
 |-- v_uf_entr_pdd: string (nullable = true)
 |-- v_lc_ent_pdd: string (nullable = true)
 |-- n_vlr_tot_pdd: decimal(38,2) (nullable = true)

-RECORD 0-----------------------------
 n_id_pdd       | 1187021679777       
 d_dt_eft_pdd   | 2023-09-13          
 d_dt_entr_pdd  | 2023-09-13 21:49:15 
 v_cnl_orig_pdd | L                   
 v_uf_entr_pdd  | RS                  
 v_lc_ent_pdd   | VIAMAO              
 n_vlr_tot_pdd  | 19.99               
only showing top 1 row



"/* 'Bronze' */\nSELECT \n        n_id_pdd,\n\td_dt_eft_pdd,\n\td_dt_entr_pdd,\n\tv_cnl_orig_pdd,\n\tv_uf_entr_pdd,\n\tv_lc_ent_pdd,\n\tn_vlr_tot_pdd, \n        BASE64(CONCAT(CAST(n_id_pdd AS STRING) , CAST(d_dt_eft_pdd AS STRING) , CAST(d_dt_entr_pdd AS STRING) , CAST(v_cnl_orig_pdd AS STRING) , CAST(v_uf_entr_pdd AS STRING) , CAST(v_lc_ent_pdd AS STRING) , CAST(n_vlr_tot_pdd AS STRING))) AS hash_id\nFROM pedidos_tmp\n"

In [5]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("pedidos_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(pd.n_id_pdd AS BIGINT) AS id_pedido,
    COALESCE(CAST(pd.d_dt_eft_pdd AS DATE), '') AS data_realizacao_pedido,
    COALESCE(date_format(pd.d_dt_entr_pdd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_entrega,
    CASE 
        WHEN pd.v_cnl_orig_pdd = 'L' THEN 'Loja'
        WHEN pd.v_cnl_orig_pdd = 'A' THEN 'App'
        WHEN pd.v_cnl_orig_pdd = 'S' THEN 'Site'
    END AS canal_venda,
    COALESCE(CAST(pd.v_uf_entr_pdd AS STRING), '') AS UF_pedido,
    COALESCE(CAST(pd.v_lc_ent_pdd AS STRING), '') AS cidade_entrega,
    CAST(pd.n_vlr_tot_pdd AS DECIMAL(38,2)) AS valor_total_pedido
    BASE64(CAST(n_id_pdd AS STRING)) AS hash_id
FROM pedidos_tmp as pd""")

spark.sql(query).show()
query

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'BASE64'.(line 12, pos 70)

== SQL ==
/* 'Silver' */
SELECT CAST(pd.n_id_pdd AS BIGINT) AS id_pedido,
       COALESCE(CAST(pd.d_dt_eft_pdd AS DATE), '') AS data_realizacao_pedido,
       COALESCE(date_format(pd.d_dt_entr_pdd, 'yyyy-MM-dd HH:mm:ss'), '') AS data_entrega,
       CASE
           WHEN pd.v_cnl_orig_pdd = 'L' THEN 'Loja'
           WHEN pd.v_cnl_orig_pdd = 'A' THEN 'App'
           WHEN pd.v_cnl_orig_pdd = 'S' THEN 'Site'
       END AS canal_venda,
       COALESCE(CAST(pd.v_uf_entr_pdd AS STRING), '') AS UF_pedido,
       COALESCE(CAST(pd.v_lc_ent_pdd AS STRING), '') AS cidade_entrega,
       CAST(pd.n_vlr_tot_pdd AS DECIMAL(38, 2)) AS valor_total_pedido BASE64(CAST(n_id_pdd AS STRING)) AS hash_id
----------------------------------------------------------------------^^^
FROM pedidos_tmp AS pd


# Pedidos Vendas

In [22]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\desafio_panvel-data_engineer\datalake\camada_0_transient\PEDIDO_VENDA"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final} 
FROM pedido_venda_tmp
"""

root
 |-- n_id_fil: long (nullable = true)
 |-- n_id_vd_fil: long (nullable = true)
 |-- n_id_pdd: long (nullable = true)

-RECORD 0--------------------
 n_id_fil    | 2408784       
 n_id_vd_fil | 34262326131   
 n_id_pdd    | 1189183932027 
only showing top 1 row



"/* 'Bronze' */\nSELECT \n        n_id_fil,\n\tn_id_vd_fil,\n\tn_id_pdd,\n        BASE64(CONCAT(CAST(n_id_fil AS STRING) , CAST(n_id_vd_fil AS STRING) , CAST(n_id_pdd AS STRING))) AS hash_id \nFROM pedido_venda_tmp\n"

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("pedido_venda_tmp")

query = formatar_sql("""/* 'Silver'*/
SELECT 
    CAST(pv.n_id_fil AS BIGINT) AS codigo_filial,
    CAST(pv.n_id_vd_fil AS BIGINT) AS id_venda_filial,
    CAST(pv.n_id_pdd AS BIGINT) AS id_pedido
    BASE64(CONCAT(CAST(n_id_fil AS STRIN), CAST(n_id_vd_fil AS STRING), CAST(n_id_pdd AS STRING))) AS hash_id
FROM pedido_venda_tmp as pv""")



spark.sql(query).show()
query

+-------------+---------------+-------------+
|codigo_filial|id_venda_filial|    id_pedido|
+-------------+---------------+-------------+
|      2408784|    34262326131|1189183932027|
|       242984|    35434048331|1189596350677|
|      2657984|     3474198431|1192614919827|
|      2517984|      409543431|1188810414807|
|      2457084|     4339980931|1192522099777|
|       229684|    35465235531|1188385018407|
|      2620184|     3789478231|1187171860557|
|      2174984|    35255133331|1186640698547|
|      2662184|     3752751431|1188256482257|
|       249284|    35437182231|1185665835127|
|      2626484|     3573693431|1192236782787|
|      2462684|    34129728831|1193157735357|
|      2471784|     4054827931|1191233879657|
|      2310784|    34374246731|1187244935557|
|      2453584|    34547578131|1184629304547|
|      2593584|     4082528131|1184834489897|
|       236684|    39168868431|1191362606727|
|      2454284|    34532390431|1193198429067|
|      2480184|     3964413431|118

"/* 'Silver'*/\nSELECT CAST(pv.n_id_fil AS BIGINT) AS codigo_filial,\n       CAST(pv.n_id_vd_fil AS BIGINT) AS id_venda_filial,\n       CAST(pv.n_id_pdd AS BIGINT) AS id_pedido\nFROM pedido_venda_tmp AS pv"

# Itens Vendas

In [21]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\desafio_panvel-data_engineer\datalake\camada_0_transient\ITENS_VENDAS"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM itens_vendas_tmp
"""


root
 |-- n_id_fil: long (nullable = true)
 |-- n_id_vd_fil: long (nullable = true)
 |-- n_id_it: long (nullable = true)
 |-- v_rc_elt: string (nullable = true)
 |-- v_it_vd_conv: string (nullable = true)
 |-- n_vlr_pis: decimal(38,2) (nullable = true)
 |-- n_vlr_vd: decimal(38,2) (nullable = true)
 |-- n_vlr_desc: decimal(38,2) (nullable = true)
 |-- n_qtd: decimal(38,4) (nullable = true)

-RECORD 0------------------
 n_id_fil     | 2326184    
 n_id_vd_fil  | 4104333431 
 n_id_it      | 433914026  
 v_rc_elt     | NAO        
 v_it_vd_conv | NAO        
 n_vlr_pis    | 0.00       
 n_vlr_vd     | 38.49      
 n_vlr_desc   | 8.50       
 n_qtd        | 1.0000     
only showing top 1 row



"/* 'Bronze' */\nSELECT \n        n_id_fil,\n\tn_id_vd_fil,\n\tn_id_it,\n\tv_rc_elt,\n\tv_it_vd_conv,\n\tn_vlr_pis,\n\tn_vlr_vd,\n\tn_vlr_desc,\n\tn_qtd,\n        BASE64(CONCAT(CAST(n_id_fil AS STRING) , CAST(n_id_vd_fil AS STRING) , CAST(n_id_it AS STRING) , CAST(v_rc_elt AS STRING) , CAST(v_it_vd_conv AS STRING) , CAST(n_vlr_pis AS STRING) , CAST(n_vlr_vd AS STRING) , CAST(n_vlr_desc AS STRING) , CAST(n_qtd AS STRING))) AS hash_id\nFROM itens_vendas_tmp\n"

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("itens_vendas_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(iv.n_id_fil AS BIGINT) AS codigo_filial,
    CAST(iv.n_id_vd_fil AS BIGINT) AS id_venda_filial,
    CAST(iv.n_id_it AS BIGINT) AS codigo_item_venda,
    CASE 
        WHEN iv.v_rc_elt  = 'SIM' THEN True
        ELSE False
    END AS com_receita_eletronica,
    CASE 
        WHEN iv.v_it_vd_conv = 'SIM' THEN 'Convênio'
        WHEN iv.v_it_vd_conv = 'NAO' AND iv.n_vlr_desc > 0 THEN 'Promoção'
        ELSE 'Sem Desconto'
    END AS tipo_desconto,
    CAST(iv.n_vlr_pis AS DECIMAL(38,2)) AS valor_pis_item,
    CAST(iv.n_vlr_vd AS DECIMAL(38,2)) AS valor_final_item,
    CAST(iv.n_vlr_desc AS DECIMAL(38,2)) AS valor_desconto_item,
    CAST(iv.n_qtd AS BIGINT) AS quantidade_itens,
FROM itens_vendas_tmp as iv""")

spark.sql(query).show()
query

+-------------+---------------+-----------------+----------------------+-------------+--------------+----------------+-------------------+----------------+
|codigo_filial|id_venda_filial|codigo_item_venda|com_receita_eletronica|tipo_desconto|valor_pis_item|valor_final_item|valor_desconto_item|quantidade_itens|
+-------------+---------------+-----------------+----------------------+-------------+--------------+----------------+-------------------+----------------+
|      2326184|     4104333431|        433914026|                 false|     Promoção|          0.00|           38.49|               8.50|               1|
|       238084|    37050030831|        405284776|                 false| Sem Desconto|          0.14|            8.49|               0.00|               1|
|      2439584|     4272478331|        403805896|                 false| Sem Desconto|          0.00|           17.99|               0.00|               1|
|      2618784|     3678370531|        502317426|               

"/* 'Silver' */\nSELECT CAST(iv.n_id_fil AS BIGINT) AS codigo_filial,\n       CAST(iv.n_id_vd_fil AS BIGINT) AS id_venda_filial,\n       CAST(iv.n_id_it AS BIGINT) AS codigo_item_venda,\n       CASE\n           WHEN iv.v_rc_elt = 'SIM' THEN TRUE\n           ELSE FALSE\n       END AS com_receita_eletronica,\n       CASE\n           WHEN iv.v_it_vd_conv = 'SIM' THEN 'Convênio'\n           WHEN iv.v_it_vd_conv = 'NAO'\n                AND iv.n_vlr_desc > 0 THEN 'Promoção'\n           ELSE 'Sem Desconto'\n       END AS tipo_desconto,\n       CAST(iv.n_vlr_pis AS DECIMAL(38, 2)) AS valor_pis_item,\n       CAST(iv.n_vlr_vd AS DECIMAL(38, 2)) AS valor_final_item,\n       CAST(iv.n_vlr_desc AS DECIMAL(38, 2)) AS valor_desconto_item,\n       CAST(iv.n_qtd AS BIGINT) AS quantidade_itens\nFROM itens_vendas_tmp AS iv"

# Endereços Clientes

In [20]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\desafio_panvel-data_engineer\datalake\camada_0_transient\ENDERECOS_CLIENTES"))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM enderecos_clientes_tmp
"""

root
 |-- v_id_cli: string (nullable = true)
 |-- n_sq_end: long (nullable = true)
 |-- d_dt_exc: string (nullable = true)
 |-- v_lcl: string (nullable = true)
 |-- v_uf: string (nullable = true)

-RECORD 0------------------------
 v_id_cli | 00D3BFA61A4EB684B1F5 
 n_sq_end | 7470026              
 d_dt_exc | null                 
 v_lcl    | CURITIBA             
 v_uf     | PR                   
only showing top 1 row



"/* 'Bronze' */\nSELECT \n        v_id_cli,\n\tn_sq_end,\n\td_dt_exc,\n\tv_lcl,\n\tv_uf,\n        BASE64(CONCAT(CAST(v_id_cli AS STRING) , CAST(n_sq_end AS STRING) , CAST(d_dt_exc AS STRING) , CAST(v_lcl AS STRING) , CAST(v_uf AS STRING))) AS hash_id\nFROM enderecos_clientes_tmp\n"

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("enderecos_clientes_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(ec.v_id_cli AS STRING) AS codigo_cliente,
    CAST(ec.n_sq_end AS BIGINT) AS sequencia_endereco_cliente,
    COALESCE(date_format(ec.d_dt_exc, 'yyyy-MM-dd HH:mm:ss'), '') AS data_exclusao_endereco,
    CAST(ec.v_lcl AS STRING) AS cidade_endereco,
    CAST(ec.v_uf AS STRING) AS UF_endereco
FROM enderecos_clientes_tmp as ec""")

spark.sql(query).show()
query

+--------------------+--------------------------+----------------------+-----------------+-----------+
|      codigo_cliente|sequencia_endereco_cliente|data_exclusao_endereco|  cidade_endereco|UF_endereco|
+--------------------+--------------------------+----------------------+-----------------+-----------+
|00D3BFA61A4EB684B1F5|                   7470026|                      |         CURITIBA|         PR|
|0313852B5434EC0A76D1|                   6772409|                      |         LONDRINA|         PR|
|04194EF6A82A8F1C519C|                   6459025|                      |SANTA CRUZ DO SUL|         RS|
|01AF764285A24153070B|                   5912571|                      |     PORTO ALEGRE|         RS|
|04A52338FAB5D10204AD|                         1|                      |         GRAVATAI|         RS|
|04D139F9FE86539E02F5|                         1|                      |         CURITIBA|         PR|
|035A054182C4F2CFF563|                         1|                      | 

"/* 'Silver' */\nSELECT CAST(ec.v_id_cli AS STRING) AS codigo_cliente,\n       CAST(ec.n_sq_end AS BIGINT) AS sequencia_endereco_cliente,\n       COALESCE(date_format(ec.d_dt_exc, 'yyyy-MM-dd HH:mm:ss'), '') AS data_exclusao_endereco,\n       CAST(ec.v_lcl AS STRING) AS cidade_endereco,\n       CAST(ec.v_uf AS STRING) AS UF_endereco\nFROM enderecos_clientes_tmp AS ec"

# Clientes Opt

In [19]:


schema = StructType([
    StructField("v_id_cli", StringType(), True),
    StructField("b_push", StringType(), True),
    StructField("b_sms", StringType(), True),
    StructField("b_email", StringType(), True),
    StructField("b_call", StringType(), True)
])

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.option("multiline","true").format("json").load(os.path.abspath(r"C:\Users\gustavo.lopes\Documentos\GitHub\desafio_panvel-data_engineer\datalake\camada_0_transient\CLIENTES_OPT"))

dados.printSchema()

dados.show(n=2, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM clientes_opt_tmp"""


root
 |-- b_call: boolean (nullable = true)
 |-- b_email: boolean (nullable = true)
 |-- b_push: boolean (nullable = true)
 |-- b_sms: boolean (nullable = true)
 |-- v_id_cli: string (nullable = true)

-RECORD 0------------------------
 b_call   | true                 
 b_email  | true                 
 b_push   | true                 
 b_sms    | true                 
 v_id_cli | 010FC87179A2C7940661 
-RECORD 1------------------------
 b_call   | null                 
 b_email  | true                 
 b_push   | null                 
 b_sms    | true                 
 v_id_cli | 04B2DB249A9623A2371B 
only showing top 2 rows



"/* 'Bronze' */\nSELECT \n        b_call,\n\tb_email,\n\tb_push,\n\tb_sms,\n\tv_id_cli,\n        BASE64(CONCAT(CAST(b_call AS STRING) , CAST(b_email AS STRING) , CAST(b_push AS STRING) , CAST(b_sms AS STRING) , CAST(v_id_cli AS STRING))) AS hash_id\nFROM clientes_opt_tmp"

In [ ]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("clientes_opt_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(co.v_id_cli AS STRING) AS codigo_cliente,
    CASE 
        WHEN co.b_push  = True THEN 'SIM'
        WHEN co.b_push  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_push,
    CASE 
        WHEN co.b_sms  = True THEN 'SIM'
        WHEN co.b_sms  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_sms,
    CASE 
        WHEN co.b_email  = True THEN 'SIM'
        WHEN co.b_email  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_email,
    CASE 
        WHEN co.b_call  = True THEN 'SIM'
        WHEN co.b_call  = False THEN 'NÃO'
        ELSE ''
    END AS autoriza_notificacao_ligacao
FROM clientes_opt_tmp as co""")

spark.sql(query).show()
query

+--------------------+-------------------------+------------------------+--------------------------+----------------------------+
|      codigo_cliente|autoriza_notificacao_push|autoriza_notificacao_sms|autoriza_notificacao_email|autoriza_notificacao_ligacao|
+--------------------+-------------------------+------------------------+--------------------------+----------------------------+
|010FC87179A2C7940661|                      SIM|                     SIM|                       SIM|                         SIM|
|04B2DB249A9623A2371B|                         |                     SIM|                       SIM|                            |
|0248D7A8AA7D752A2AF9|                      NÃO|                     SIM|                       SIM|                         SIM|
|02ABD1DDB59E705FCE49|                      NÃO|                     NÃO|                       NÃO|                         NÃO|
|050C08C7789F8BF18F10|                         |                        |                 

"/* 'Silver' */\nSELECT CAST(co.v_id_cli AS STRING) AS codigo_cliente,\n       CASE\n           WHEN co.b_push = TRUE THEN 'SIM'\n           WHEN co.b_push = FALSE THEN 'NÃO'\n           ELSE ''\n       END AS autoriza_notificacao_push,\n       CASE\n           WHEN co.b_sms = TRUE THEN 'SIM'\n           WHEN co.b_sms = FALSE THEN 'NÃO'\n           ELSE ''\n       END AS autoriza_notificacao_sms,\n       CASE\n           WHEN co.b_email = TRUE THEN 'SIM'\n           WHEN co.b_email = FALSE THEN 'NÃO'\n           ELSE ''\n       END AS autoriza_notificacao_email,\n       CASE\n           WHEN co.b_call = TRUE THEN 'SIM'\n           WHEN co.b_call = FALSE THEN 'NÃO'\n           ELSE ''\n       END AS autoriza_notificacao_ligacao\nFROM clientes_opt_tmp AS co"

# Clientes

In [18]:

# Carregando o dataframe para verificar a estrutura das colunas e tipos de dados
dados = spark.read.format("parquet").load(os.path.abspath(os.path.join(os.getcwd(), "datalake/camada_0_transient/CLIENTES")))

dados.printSchema()

dados.show(n=1, vertical=True)
colunas = ',\n\t'.join(dados.columns)
base = ['CAST({} AS STRING)'.format(col) for col in dados.columns]
base_media = ' , '.join(base)
base_final = f"BASE64(CONCAT({base_media})) AS hash_id"
base_final
print(dumps(f"""/* 'Bronze' */
SELECT 
        {colunas},
        {base_final}
FROM clientes_tmp"""))


root
 |-- v_id_cli: string (nullable = true)
 |-- d_dt_nasc: date (nullable = true)
 |-- v_sx_cli: string (nullable = true)
 |-- n_est_cvl: string (nullable = true)

-RECORD 0-------------------------
 v_id_cli  | 0000009DB36F622B7639 
 d_dt_nasc | 2003-11-14           
 v_sx_cli  | F                    
 n_est_cvl | null                 
only showing top 1 row

"/* 'Bronze' */\nSELECT \n        v_id_cli,\n\td_dt_nasc,\n\tv_sx_cli,\n\tn_est_cvl,\n        BASE64(CONCAT(CAST(v_id_cli AS STRING) , CAST(d_dt_nasc AS STRING) , CAST(v_sx_cli AS STRING) , CAST(n_est_cvl AS STRING))) AS hash_id\nFROM clientes_tmp"


In [9]:
# Criando a tabela temporária e realizando a consulta que depois usaremos para transformar a camada raw
dados.createOrReplaceTempView("clientes_tmp")

query = formatar_sql("""/* 'Silver' */
SELECT 
    CAST(c.v_id_cli AS STRING) AS codigo_cliente,
    COALESCE(CAST(c.d_dt_nasc AS DATE), '') AS data_nascimento_cliente,
    CASE 
        WHEN c.v_sx_cli  = 'F' THEN 'Feminino'
        WHEN c.v_sx_cli  = 'M' THEN 'Masculino'
        ELSE '' 
    END AS genero_biologico_cliente,
    CASE 
        WHEN c.n_est_cvl  = 1 THEN 'Solteiro'
        WHEN c.n_est_cvl  = 2 THEN 'Casado'
        WHEN c.n_est_cvl  = 3 THEN 'Viúvo'
        WHEN c.n_est_cvl  = 4 THEN 'Desquitado'
        WHEN c.n_est_cvl  = 5 THEN 'Divorciado'
        WHEN c.n_est_cvl  = 6 THEN 'Outros'
        ELSE ''
    END AS estado_civil_cliente
FROM clientes_tmp as c""")
spark.sql(query).show()
query

+--------------------+-----------------------+------------------------+--------------------+--------------------+
|      codigo_cliente|data_nascimento_cliente|genero_biologico_cliente|estado_civil_cliente|             hash_id|
+--------------------+-----------------------+------------------------+--------------------+--------------------+
|0000009DB36F622B7639|             2003-11-14|                Feminino|                    |                null|
|000000F51C15031D708E|             2008-03-23|                Feminino|                    |                null|
|00000F54BE2BBF0E7B13|             1986-02-24|                Feminino|                    |                null|
|000013E1FB44D9A9E50F|             1984-12-13|               Masculino|                    |                null|
|00001522AD94645C7688|             1992-11-10|                Feminino|              Casado|MDAwMDE1MjJBRDk0N...|
|0000161185110EE1BBE5|             1975-12-27|                Feminino|                 

"/* 'Silver' */\nSELECT CAST(c.v_id_cli AS STRING) AS codigo_cliente,\n       COALESCE(CAST(c.d_dt_nasc AS DATE), '') AS data_nascimento_cliente,\n       CASE\n           WHEN c.v_sx_cli = 'F' THEN 'Feminino'\n           WHEN c.v_sx_cli = 'M' THEN 'Masculino'\n           ELSE ''\n       END AS genero_biologico_cliente,\n       CASE\n           WHEN c.n_est_cvl = 1 THEN 'Solteiro'\n           WHEN c.n_est_cvl = 2 THEN 'Casado'\n           WHEN c.n_est_cvl = 3 THEN 'Viúvo'\n           WHEN c.n_est_cvl = 4 THEN 'Desquitado'\n           WHEN c.n_est_cvl = 5 THEN 'Divorciado'\n           WHEN c.n_est_cvl = 6 THEN 'Outros'\n           ELSE ''\n       END AS estado_civil_cliente,\n       base64(concat(*)) AS hash_id\nFROM clientes_tmp AS c"

In [ ]:
""" -- 'Tabela Endereços Clientes parquet'
SELECT 
    CAST(c.v_id_cli AS STRING) AS codigo_cliente,
    COALESCE(CAST(c.d_dt_nasc AS DATE), '') AS data_nascimento_cliente,
    CASE 
        WHEN c.v_sx_cli  = 'F' THEN 'Feminino'
        WHEN c.v_sx_cli  = 'M' THEN 'Masculino'
        ELSE '' 
    END AS genero_biologico_cliente,
    CASE 
        WHEN c.n_est_cvl  = 1 THEN 'Solteiro'
        WHEN c.n_est_cvl  = 2 THEN 'Casado'
        WHEN c.n_est_cvl  = 3 THEN 'Viúvo'
        WHEN c.n_est_cvl  = 4 THEN 'Desquitado'
        WHEN c.n_est_cvl  = 5 THEN 'Divorciado'
        WHEN c.n_est_cvl  = 6 THEN 'Outros'
        ELSE ''
    END AS estado_civil_cliente
FROM clientes_tmp as c"""

" -- 'Tabela Endereços Clientes parquet'\nSELECT \n    CAST(c.v_id_cli AS STRING) AS codigo_cliente,\n    COALESCE(CAST(c.d_dt_nasc AS DATE), '') AS data_nascimento_cliente,\n    CASE \n        WHEN c.v_sx_cli  = 'F' THEN 'Feminino'\n        WHEN c.v_sx_cli  = 'M' THEN 'Masculino'\n        ELSE '' \n    END AS genero_biologico_cliente,\n    CASE \n        WHEN c.n_est_cvl  = 1 THEN 'Solteiro'\n        WHEN c.n_est_cvl  = 2 THEN 'Casado'\n        WHEN c.n_est_cvl  = 3 THEN 'Viúvo'\n        WHEN c.n_est_cvl  = 4 THEN 'Desquitado'\n        WHEN c.n_est_cvl  = 5 THEN 'Divorciado'\n        WHEN c.n_est_cvl  = 6 THEN 'Outros'\n        ELSE ''\n    END AS estado_civil_cliente\nFROM clientes_tmp as c"

In [ ]:
""" /* Gold 'Vendas' */
SELECT 
    v.codigo_filial AS codifo_filial,
    v.id_venda_filial AS codigo_cupom_venda,
    v.data_emissao AS data_emissao,
    iv.codigo_item_venda AS codigo_item,
    iv.valor_final_item AS valor_unitario,
    iv.quantidade_itens AS quantidade,
    v.codigo_cliente AS codigo_cliente,
    iv.tipo_desconto AS tipo_desconto,
    p.canal_venda AS canal_venda
FROM vendas AS v
INNER JOIN pedido_venda AS pv ON CONCAT(CAST(pv.codigo_filial AS STRING), CAST(pv.id_venda_filial AS STRING)) = CONCAT(CAST(v.codigo_filial AS STRING), CAST(v.id_venda_filial AS STRING))
INNER JOIN itens_vendas AS iv ON CONCAT(CAST(iv.codigo_filial AS STRING), CAST(iv.id_venda_filial)) = CONCAT(CAST(v.codigo_filial AS STRING), CAST(v.id_venda_filial AS STRING))
INNER JOIN pedidos AS p ON p.id_pedido = pv.id_pedido
"""

" /* Gold 'Vendas' */\nSELECT \n    v.codigo_filial AS codifo_filial,\n    v.id_venda_filial AS codigo_cupom_venda,\n    v.data_emissao AS data_emissao,\n    iv.codigo_item_venda AS codigo_item,\n    iv.valor_final_item AS valor_unitario,\n    iv.quantidade_itens AS quantidade,\n    v.codigo_cliente AS codigo_cliente,\n    iv.tipo_desconto AS tipo_desconto,\n    p.canal_venda AS canal_venda\nFROM vendas AS v\nINNER JOIN pedido_venda AS pv ON CONCAT(CAST(pv.codigo_filial AS STRING), CAST(pv.id_venda_filial AS STRING)) = CONCAT(CAST(v.codigo_filial AS STRING), CAST(v.id_venda_filial AS STRING))\nINNER JOIN itens_vendas AS iv ON CONCAT(CAST(iv.codigo_filial AS STRING), CAST(iv.id_venda_filial)) = CONCAT(CAST(v.codigo_filial AS STRING), CAST(v.id_venda_filial AS STRING))\nINNER JOIN pedidos AS p ON p.id_pedido = pv.id_pedido\n"

In [ ]:
"""/* Gold 'Clientes' */
SELECT
    c.codigo_cliente AS codigo_cliente,
    c.data_nascimento_cliente AS data_nascimento,
    datediff(current_date(), CAST(data_nascimento_cliente AS DATE)) AS idade,
    c.genero_biologico_cliente AS sexo,
    c.UF_endereco AS uf,
    ec.cidade_endereco AS cidade,
    c.estado_civil_cliente AS estado_civil,
    co.autoriza_notificacao_ligacao AS flag_lgpd_call,
    co.autoriza_notificacao_sms AS flag_lgpd_sms,
    co.autoriza_notificacao_email AS flag_lgpd_email,
    co.autoriza_notificacao_push AS flag_lgpd_push
FROM clientes AS c
INNER JOIN clientes_opt AS co ON co.codigo_cliente = c.codigo_cliente
INNER JOIN enderecos_clientes AS ec ON ec.codigo_cliente = c.codigo_cliente"""

"/* Gold 'Clientes' */\nSELECT\n    c.codigo_cliente AS codigo_cliente,\n    c.data_nascimento_cliente AS data_nascimento,\n    datediff(current_date(), CAST(data_nascimento_cliente AS DATE)) AS idade,\n    c.genero_biologico_cliente AS sexo,\n    c.UF_endereco AS uf,\n    ec.cidade_endereco AS cidade,\n    c.estado_civil_cliente AS estado_civil,\n    co.autoriza_notificacao_ligacao AS flag_lgpd_call,\n    co.autoriza_notificacao_sms AS flag_lgpd_sms,\n    co.autoriza_notificacao_email AS flag_lgpd_email,\n    co.autoriza_notificacao_push AS flag_lgpd_push\nFROM clientes AS c\nINNER JOIN clientes_opt AS co ON co.codigo_cliente = c.codigo_cliente\nINNER JOIN enderecos_clientes AS ec ON ec.codigo_cliente = c.codigo_cliente"